In [ ]:
!pip install --upgrade transformers

In [ ]:
import transformers
from transformers import Trainer, TrainingArguments

In [ ]:
# 질의 응답 모델 가져오기 (러시아어 기반 다국어 모델)
# Load QA model (Russian-based multilingual model)
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

model_name = "AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/781 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/516 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 498.5 kB/s eta 0:00:00


In [ ]:
import collections
import numpy as np
import evaluate

metric = evaluate.load("squad")

In [ ]:
# 모델 구조 확인
# Inspect structure of the model
model.qa_outputs

Linear(in_features=1024, out_features=2, bias=True)

In [ ]:
# 모델 층명 확인
# Check model layer names
list(model.named_parameters())

[('roberta.embeddings.word_embeddings.weight',
  Parameter containing:
  tensor([[-0.0560, -0.0291, -0.0792,  ...,  0.0294,  0.0246, -0.0303],
          [-0.0042,  0.0026,  0.0078,  ..., -0.0043,  0.0066, -0.0057],
          [-0.0156,  0.0124, -0.1050,  ...,  0.1184,  0.1372,  0.2036],
          ...,
          [ 0.1801,  0.1886,  0.2450,  ..., -0.0121,  0.1840, -0.1537],
          [ 0.0573, -0.0343, -0.0763,  ...,  0.0725,  0.0668,  0.0678],
          [ 0.0263,  0.0167, -0.0438,  ...,  0.0115,  0.0864,  0.0685]],
         requires_grad=True)),
 ('roberta.embeddings.position_embeddings.weight',
  Parameter containing:
  tensor([[-0.0004,  0.0109,  0.0033,  ...,  0.0049, -0.0419,  0.0006],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0603,  0.0130, -0.0638,  ..., -0.1246, -0.2179, -0.1939],
          ...,
          [ 0.0262,  0.0647,  0.0959,  ...,  0.0457, -0.2124, -0.1015],
          [ 0.0086,  0.1375,  0.0471,  ...,  0.0323, -0.2461, -0.0995],


In [ ]:
# 다르게 학습 적용할 동일모델 가져오기
# Load the same model for alternative training
model2 = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer2 = AutoTokenizer.from_pretrained(model_name)

In [ ]:
model3 = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer3 = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# 각 모델 학습방향 설정
# 비교위해 세모델 모두 동일한 한글 데이터셋으로 학습
# model: 출력층만 학습, 나머지층 freeze
# model2: 출력층 + encoder 마지막 layer2개층 학습
# model3: 전체 파라미터 파인튜닝

# Set training strategy for each model
# All three models trained on the same Korean dataset for comparison
# model: only output layer is trainable, rest are frozen
# model2: output layer + last two encoder layers are trainable
# model3: full fine-tuning of all parameters

In [ ]:
# 파라미터 적용여부 함수 생성
# Create function to set trainable parameters

def set_trainable_layers(model, mode="head_only"):
  if mode == "head_only":
    for name, param in model.named_parameters():
      if "qa_outputs" in name:
        param.requires_grad = True
      else:
        param.requires_grad = False


  elif mode == "head_top2":
    for name, param in model.named_parameters():
      if "qa_outputs" in name or "layer.22" in name or "layer.23" in name:
        param.requires_grad = True
      else:
        param.requires_grad = False


  elif mode == "full":
    for param in model.parameters():
      param.requires_grad = True


  else:
    raise ValueError("mode should be 'head_only', 'head_top2', or 'full'")

In [ ]:
# 각 모델의 의도에 맞게 함수 적용
# Apply the parameter-setting function according to each model's training strategy
set_trainable_layers(model, mode="head_only")
set_trainable_layers(model2, mode="head_top2")
set_trainable_layers(model3, mode="full")

In [ ]:
# 함수적용여부 레이어 확인
# Verify which layers have the function applied
for name, param in model.named_parameters():
  print(name, param.requires_grad)

roberta.embeddings.word_embeddings.weight False
roberta.embeddings.position_embeddings.weight False
roberta.embeddings.token_type_embeddings.weight False
roberta.embeddings.LayerNorm.weight False
roberta.embeddings.LayerNorm.bias False
roberta.encoder.layer.0.attention.self.query.weight False
roberta.encoder.layer.0.attention.self.query.bias False
roberta.encoder.layer.0.attention.self.key.weight False
roberta.encoder.layer.0.attention.self.key.bias False
roberta.encoder.layer.0.attention.self.value.weight False
roberta.encoder.layer.0.attention.self.value.bias False
roberta.encoder.layer.0.attention.output.dense.weight False
roberta.encoder.layer.0.attention.output.dense.bias False
roberta.encoder.layer.0.attention.output.LayerNorm.weight False
roberta.encoder.layer.0.attention.output.LayerNorm.bias False
roberta.encoder.layer.0.intermediate.dense.weight False
roberta.encoder.layer.0.intermediate.dense.bias False
roberta.encoder.layer.0.output.dense.weight False
roberta.encoder.layer.

In [ ]:
for name, param in model2.named_parameters():
  print(name, param.requires_grad)

roberta.embeddings.word_embeddings.weight False
roberta.embeddings.position_embeddings.weight False
roberta.embeddings.token_type_embeddings.weight False
roberta.embeddings.LayerNorm.weight False
roberta.embeddings.LayerNorm.bias False
roberta.encoder.layer.0.attention.self.query.weight False
roberta.encoder.layer.0.attention.self.query.bias False
roberta.encoder.layer.0.attention.self.key.weight False
roberta.encoder.layer.0.attention.self.key.bias False
roberta.encoder.layer.0.attention.self.value.weight False
roberta.encoder.layer.0.attention.self.value.bias False
roberta.encoder.layer.0.attention.output.dense.weight False
roberta.encoder.layer.0.attention.output.dense.bias False
roberta.encoder.layer.0.attention.output.LayerNorm.weight False
roberta.encoder.layer.0.attention.output.LayerNorm.bias False
roberta.encoder.layer.0.intermediate.dense.weight False
roberta.encoder.layer.0.intermediate.dense.bias False
roberta.encoder.layer.0.output.dense.weight False
roberta.encoder.layer.

In [ ]:
for name, name in model3.named_parameters():
  print(name, param.requires_grad)

Parameter containing:
tensor([[-0.0560, -0.0291, -0.0792,  ...,  0.0294,  0.0246, -0.0303],
        [-0.0042,  0.0026,  0.0078,  ..., -0.0043,  0.0066, -0.0057],
        [-0.0156,  0.0124, -0.1050,  ...,  0.1184,  0.1372,  0.2036],
        ...,
        [ 0.1801,  0.1886,  0.2450,  ..., -0.0121,  0.1840, -0.1537],
        [ 0.0573, -0.0343, -0.0763,  ...,  0.0725,  0.0668,  0.0678],
        [ 0.0263,  0.0167, -0.0438,  ...,  0.0115,  0.0864,  0.0685]],
       requires_grad=True) True
Parameter containing:
tensor([[-0.0004,  0.0109,  0.0033,  ...,  0.0049, -0.0419,  0.0006],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0603,  0.0130, -0.0638,  ..., -0.1246, -0.2179, -0.1939],
        ...,
        [ 0.0262,  0.0647,  0.0959,  ...,  0.0457, -0.2124, -0.1015],
        [ 0.0086,  0.1375,  0.0471,  ...,  0.0323, -0.2461, -0.0995],
        [ 0.0624,  0.2539,  0.0526,  ...,  0.0461, -0.2437, -0.0828]],
       requires_grad=True) True
Parameter containing:
te

In [ ]:
# 데이터셋 불러오기
# Load dataset
from transformers import Trainer, TrainingArguments
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")

README.md: 0.00B [00:00, ?B/s]

squad_kor_v1/train-00000-of-00001.parque(…):   0%|          | 0.00/11.6M [00:00<?, ?B/s]

squad_kor_v1/validation-00000-of-00001.p(…):   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})

In [ ]:
print(transformers.__version__)

4.57.1


In [ ]:
from transformers.trainer_utils import EvalPrediction

def prepare_train_features(examples):
    # context + question 토크나이징
    # context + question tokenizing
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]

        # 정답이 없으면 CLS 토큰
        # Use CLS token when there is no label
        if len(answers["answer_start"]) == 0:
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0
            token_end_index = len(input_ids) - 1

            # context token 위치 찾기
            # Find positions of context tokens
            while tokenized_examples.sequence_ids(i)[token_start_index] != 1:
                token_start_index += 1
            while tokenized_examples.sequence_ids(i)[token_end_index] != 1:
                token_end_index -= 1

            # 답이 context 밖이면 CLS 토큰
            # Use CLS token if the answer is outside the context
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                start_positions.append(cls_index)
                end_positions.append(cls_index)
            else:
                # 토큰 단위 start/end 계산
                # Calculate start and end positions at the token level
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                start_positions.append(token_start_index - 1)

                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                end_positions.append(token_end_index + 1)

    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions
    return tokenized_examples

# 검증 데이터 특징 준비 함수 (ID와 오프셋 저장)
# Function to prepare validation features (store IDs and offsets)
def prepare_validation_features(examples):
    tokenized_examples = tokenizer(
        examples["question"], examples["context"],
        truncation="only_second", max_length=384,
        stride=128, return_overflowing_tokens=True,
        return_offsets_mapping=True, padding="max_length"
    )
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    tokenized_examples["example_id"] = []
    tokenized_examples["sequence_ids"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])
        tokenized_examples["sequence_ids"].append(tokenized_examples.sequence_ids(i))

    return tokenized_examples

# 훈련 데이터 전처리 적용
# Apply preprocessing to the training data
tokenized_datasets = dataset.map(
    prepare_train_features,
    batched=True,
    remove_columns=dataset["train"].column_names
)

# 검증 데이터 전처리 및 원본 데이터 저장
# Preprocess validation data and save original copies
validation_features = dataset["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=dataset["validation"].column_names
)
validation_examples = dataset["validation"]

# SQuAD 후처리 함수: Logits -> 텍스트 답변 추출
# SQuAD post-processing function: convert logits to text answers
def post_processing_function(examples, features, raw_predictions, n_best_size=20, max_answer_length=30):
    start_logits, end_logits = raw_predictions
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    all_predictions = collections.OrderedDict()

    for example_index, example in enumerate(examples):
        feature_indices = features_per_example[example_index]
        valid_answers = []
        context = example["context"]

        for feature_index in feature_indices:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            sequence_ids = features[feature_index]['sequence_ids']
            context_index = 1 # 컨텍스트의 sequence ID Context sequence ID

            starts = np.argsort(start_logit)[-1: -n_best_size - 1: -1].tolist()
            ends = np.argsort(end_logit)[-1: -n_best_size - 1: -1].tolist()

            for start in starts:
                for end in ends:
                    if sequence_ids[start] != context_index or sequence_ids[end] != context_index: continue
                    if end < start: continue
                    length = end - start + 1
                    if length > max_answer_length: continue

                    start_char = offsets[start][0]
                    end_char = offsets[end][1]
                    answer_text = context[start_char: end_char]

                    valid_answers.append({
                        "score": start_logit[start] + end_logit[end],
                        "text": answer_text
                    })

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
            prediction = best_answer["text"]
        else:
            prediction = ""

        all_predictions[example["id"]] = prediction

    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    pred_list = [{"id": k, "prediction_text": v} for k, v in all_predictions.items()]
    ref_list = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return pred_list, ref_list

# Trainer에 전달할 compute_metrics 함수
# compute_metrics function to be passed to the Trainer
def compute_metrics(p: EvalPrediction):
    # 후처리 실행: (원본 예제, 특징, 예측 결과)를 전달
    # Execute post-processing: pass original examples, features, and predictions
    predictions, references = post_processing_function(
        validation_examples,
        validation_features,
        p.predictions
    )
    result = metric.compute(predictions=predictions, references=references)

    return {"eval_f1": result["f1"], "eval_exact_match": result["exact_match"]}

# 트레이닝 세팅
# Training setting
training_args = TrainingArguments(
    output_dir="./qa_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",   # compute_metrics가 반환하는 키 'f1' 사용 Use the 'f1' key returned by compute_metrics
    greater_is_better=True,
    fp16=True,
    logging_steps=100,
    report_to="none"
)

# Trainer 정의
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=validation_features, # 토크나이징된 Features를 평가 데이터셋으로 사용 Use tokenized features as the evaluation dataset
    compute_metrics=compute_metrics    # F1 계산 함수 전달 Pass the F1 computation function
)

trainer2 = Trainer(
    model=model2,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=validation_features,
    compute_metrics=compute_metrics
)

trainer3 = Trainer(
    model=model3,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=validation_features,
    compute_metrics=compute_metrics
)

Map:   0%|          | 0/60407 [00:00<?, ? examples/s]

Map:   0%|          | 0/5774 [00:00<?, ? examples/s]

Map:   0%|          | 0/5774 [00:00<?, ? examples/s]

In [ ]:
# 학습
# Training
trainer.train()
trainer2.train()
trainer3.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
